In [2]:
import json

In [1]:
keywords = [
    "Machine Learning",
    "Deep Learning", 
    "Artificial Intelligence",
    "AI",
    "Artificial General Intelligence",
    "AGI",
    "Large Language Models",
    "LLMs", "VLMs",
    "Generative Models",
    "Natural Language Processing",
    "NLP",
    "Computer Vision",
    "Reinforcement Learning",
    "Big Data",
    "Data Mining",
    "Explainable AI",
    "XAI",
    "Interpretable Machine Learning",
    "Graph Neural Networks",
    "Bayesian Inference",
    "Causal Inference",
    "Causal Discovery",
    "Foundation Models",
    "Self-Supervised Learning",
    "Multi-Agent Systems",
    "Ethics and Bias in AI"
]
filenames = [f'/Users/wwy/Documents/UCSD/DSC202/final_project/semanticscholar_data/{keyword}.json' for keyword in keywords]

In [6]:
with open(filenames[0], 'r') as f:
    data = json.load(f)['data']
    print(data[0].keys())
    print(len(data[0]['citations']))

dict_keys(['paperId', 'title', 'citationCount', 'citations'])
1000


In [16]:
import json
import pandas as pd
from pathlib import Path

def combine_json_files(json_directory):
    # Prepare containers for database structures
    all_papers = {}  # Using dict to ensure paper uniqueness
    citations = []
    keywords = []
    paper_keywords = []
    authors = {}  # Using dict with authorId as key
    paper_authors = []
    
    def process_authors(paper_id, author_list):
        paper_author_rels = []
        for author in author_list:
            author_id = author.get('authorId')
            author_name = author.get('name')
            
            if not author_id or not author_name:
                continue
                
            # Add author if not already added
            if author_id not in authors:
                authors[author_id] = {
                    'author_id': author_id,
                    'name': author_name
                }
            
            paper_author_rels.append({
                'paper_id': paper_id,
                'author_id': author_id
            })
        return paper_author_rels
    
    # Process each JSON file
    for json_file in Path(json_directory).glob('*.json'):
        keyword = json_file.stem
        keywords.append({
            'keyword_id': len(keywords) + 1,
            'keyword': keyword
        })
        keyword_id = len(keywords)
        
        # Load JSON data
        with open(json_file, 'r', encoding='utf-8') as f:
            papers = json.load(f)['data']
            
        # Process each paper
        for paper in papers:
            paper_id = paper.get('paperId')
            
            # Add paper if not already added
            if paper_id not in all_papers:
                all_papers[paper_id] = {
                    'paper_id': paper_id,
                    'title': paper.get('title'),
                    'citation_count': paper.get('citationCount')
                }
                
                # Process authors for main paper
                paper_authors.extend(process_authors(paper_id, paper.get('authors', [])))
            
            # Add paper-keyword relationship
            paper_keywords.append({
                'paper_id': paper_id,
                'keyword_id': keyword_id
            })
            
            # Process citations
            for citation in paper.get('citations', []):
                cited_paper_id = citation.get('paperId')
                
                # Add cited paper if new
                if cited_paper_id not in all_papers:
                    all_papers[cited_paper_id] = {
                        'paper_id': cited_paper_id,
                        'title': citation.get('title'),
                        'citation_count': citation.get('citationCount')
                    }
                    
                    # Process authors for cited paper
                    paper_authors.extend(process_authors(cited_paper_id, citation.get('authors', [])))
                
                # Add citation relationship
                citations.append({
                    'cited_paper_id': paper_id,
                    'citing_paper_id': cited_paper_id
                })
    
    return {
        'papers': list(all_papers.values()),
        'citations': citations,
        'keywords': keywords,
        'paper_keywords': paper_keywords,
        'authors': list(authors.values()),
        'paper_authors': paper_authors
    }

In [13]:
def save_for_databases(combined_data, output_dir='/Users/wwy/Documents/UCSD/DSC202/final_project/database_files'):
    # Create output directories
    sql_dir = Path(output_dir) / 'sql'
    neo4j_dir = Path(output_dir) / 'neo4j'
    sql_dir.mkdir(parents=True, exist_ok=True)
    neo4j_dir.mkdir(parents=True, exist_ok=True)
    
    # Save SQL-ready files
    pd.DataFrame(combined_data['papers']).to_csv(
        sql_dir / 'papers.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['citations']).to_csv(
        sql_dir / 'citations.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['keywords']).to_csv(
        sql_dir / 'keywords.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['paper_keywords']).to_csv(
        sql_dir / 'paper_keywords.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['authors']).to_csv(
        sql_dir / 'authors.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['paper_authors']).to_csv(
        sql_dir / 'paper_authors.csv',
        index=False,
        encoding='utf-8'
    )
    
    # Save Neo4j-ready files
    pd.DataFrame(combined_data['papers']).to_csv(
        neo4j_dir / 'papers.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['citations']).to_csv(
        neo4j_dir / 'citations.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['authors']).to_csv(
        neo4j_dir / 'authors.csv',
        index=False,
        encoding='utf-8'
    )
    
    pd.DataFrame(combined_data['paper_authors']).to_csv(
        neo4j_dir / 'paper_authors.csv',
        index=False,
        encoding='utf-8'
    )
    
    # Keywords and relationships for Neo4j
    keyword_relationships = []
    for pk in combined_data['paper_keywords']:
        keyword = next(k['keyword'] for k in combined_data['keywords'] 
                      if k['keyword_id'] == pk['keyword_id'])
        keyword_relationships.append({
            'paper_id': pk['paper_id'],
            'keyword': keyword
        })
    
    pd.DataFrame(keyword_relationships).to_csv(
        neo4j_dir / 'keyword_relationships.csv',
        index=False,
        encoding='utf-8'
    )

In [14]:
def generate_database_scripts(output_dir):
    # SQL creation script
    sql_script = """
CREATE TABLE papers (
    paper_id VARCHAR(255) PRIMARY KEY,
    title TEXT NOT NULL,
    citation_count INTEGER,
    abstract TEXT
);

CREATE TABLE authors (
    author_id VARCHAR(255) PRIMARY KEY,  -- Changed to VARCHAR for semantic scholar authorId
    name VARCHAR(255) NOT NULL
);

CREATE TABLE paper_authors (
    paper_id VARCHAR(255),
    author_id VARCHAR(255),  -- Changed to VARCHAR
    PRIMARY KEY (paper_id, author_id),
    FOREIGN KEY (paper_id) REFERENCES papers(paper_id),
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
);

CREATE TABLE keywords (
    keyword_id INTEGER PRIMARY KEY,
    keyword VARCHAR(255) NOT NULL UNIQUE
);

CREATE TABLE paper_keywords (
    paper_id VARCHAR(255),
    keyword_id INTEGER,
    PRIMARY KEY (paper_id, keyword_id),
    FOREIGN KEY (paper_id) REFERENCES papers(paper_id),
    FOREIGN KEY (keyword_id) REFERENCES keywords(keyword_id)
);

CREATE TABLE citations (
    citing_paper_id VARCHAR(255),
    cited_paper_id VARCHAR(255),
    PRIMARY KEY (citing_paper_id, cited_paper_id),
    FOREIGN KEY (citing_paper_id) REFERENCES papers(paper_id),
    FOREIGN KEY (cited_paper_id) REFERENCES papers(paper_id)
);

CREATE INDEX idx_papers_citation_count ON papers(citation_count);
CREATE INDEX idx_papers_title ON papers(title);
CREATE INDEX idx_authors_name ON authors(name);
CREATE INDEX idx_keywords_keyword ON keywords(keyword);
"""

    # Neo4j creation script
    neo4j_script = """
// Create constraints
CREATE CONSTRAINT paper_id IF NOT EXISTS ON (p:Paper) ASSERT p.paper_id IS UNIQUE;
CREATE CONSTRAINT author_id IF NOT EXISTS ON (a:Author) ASSERT a.author_id IS UNIQUE;
CREATE CONSTRAINT keyword_name IF NOT EXISTS ON (k:Keyword) ASSERT k.name IS UNIQUE;

// Load papers
LOAD CSV WITH HEADERS FROM 'file:///papers.csv' AS row
CREATE (p:Paper {
    paper_id: row.paper_id,
    title: row.title,
    citation_count: toInteger(row.citation_count),
    abstract: row.abstract
});

// Load authors
LOAD CSV WITH HEADERS FROM 'file:///authors.csv' AS row
CREATE (a:Author {
    author_id: row.author_id,
    name: row.name
});

// Load paper-author relationships
LOAD CSV WITH HEADERS FROM 'file:///paper_authors.csv' AS row
MATCH (p:Paper {paper_id: row.paper_id})
MATCH (a:Author {author_id: row.author_id})
CREATE (p)-[:AUTHORED_BY]->(a);

// Load citations
LOAD CSV WITH HEADERS FROM 'file:///citations.csv' AS row
MATCH (citing:Paper {paper_id: row.citing_paper_id})
MATCH (cited:Paper {paper_id: row.cited_paper_id})
CREATE (citing)-[:CITES]->(cited);

// Load keywords and relationships
LOAD CSV WITH HEADERS FROM 'file:///keyword_relationships.csv' AS row
MATCH (p:Paper {paper_id: row.paper_id})
MERGE (k:Keyword {name: row.keyword})
CREATE (p)-[:HAS_KEYWORD]->(k);

// Create indexes
CREATE INDEX paper_citation_idx IF NOT EXISTS FOR (p:Paper) ON (p.citation_count);
CREATE INDEX paper_title_idx IF NOT EXISTS FOR (p:Paper) ON (p.title);
CREATE INDEX author_name_idx IF NOT EXISTS FOR (a:Author) ON (a.name);
"""

    Path(output_dir).mkdir(parents=True, exist_ok=True)
    with open(Path(output_dir) / 'sql' / 'create_tables.sql', 'w') as f:
        f.write(sql_script)
    with open(Path(output_dir) / 'neo4j' / 'import_data.cypher', 'w') as f:
        f.write(neo4j_script)

In [17]:
json_dir = "/Users/wwy/Documents/UCSD/DSC202/final_project/semanticscholar_json"
output_dir = "/Users/wwy/Documents/UCSD/DSC202/final_project/database_files"

combined_data = combine_json_files(json_dir)
save_for_databases(combined_data, output_dir)
generate_database_scripts(output_dir)

print("Files processed and saved successfully!")
print(f"Total papers: {len(combined_data['papers'])}")
print(f"Total citations: {len(combined_data['citations'])}")
print(f"Total authors: {len(combined_data['authors'])}")
print(f"Total keywords: {len(combined_data['keywords'])}")

Files processed and saved successfully!
Total papers: 130740
Total citations: 168511
Total authors: 477314
Total keywords: 27
